In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
train_lines_in = "../data/raw/train_lines.geojson"
train_stations_in = "../data/interim/train_stations.geojson"
bus_routes_file_in = "../data/raw/bus_routes.geojson"
bus_stops_file_in = "../data/interim/bus_stops.geojson"
bike_stations_file_in = "../data/interim/bike_stations_gbfs_v2.geojson"
tract_file_in = "../data/interim/tracts.geoparquet"
comm_file_in =  "../data/interim/communities.geojson"

train_rides_in = "../data/interim/train_rides.csv"
bus_rides_in = "../data/interim/bus_rides.csv"
bike_rides_in = "../data/interim/bike_rides_v3.geoparquet"
uber_rides_in = "../data/interim/uber_rides.parquet"

train_line_rides_out = "../data/final/train_line_rides.parquet"
train_station_rides_out = "../data/final/train_station_rides.parquet"
bus_rides_out = "../data/final/bus_rides.parquet"
bike_rides_out = "../data/final/bike_rides.geoparquet"
uber_rides_out = "../data/final/uber_rides.parquet"

# Pipeline in

In [3]:
train_lines = gpd.read_file(train_lines_in)
train_stations = gpd.read_file(train_stations_in)
bus_routes = gpd.read_file(bus_routes_file_in)
bus_stops = gpd.read_file(bus_stops_file_in)
tracts = gpd.read_parquet(tract_file_in)
comms = gpd.read_file(comm_file_in)
bike_stations = gpd.read_file(bike_stations_file_in)

In [4]:
train_rides = pd.read_csv(train_rides_in)
bus_rides = pd.read_csv(bus_rides_in)
bike_rides = gpd.read_parquet(bike_rides_in)
uber_rides = pd.read_parquet(uber_rides_in)

# Train Rides + Stations

In [5]:
# We only care about granularity of entrypoints, not line/direction because we dont know
# which line/direction the riders go.
train_stations = train_stations.drop_duplicates('map_id')

In [6]:
train_rides = train_stations.merge(train_rides, how='right', right_on='station_id', left_on='map_id', validate="1:m")

# Train Rides + Lines

In [7]:
# Aggregate station features per line:
feature_cols = ['airport'] + list(train_stations.filter(regex=r'\d+m$').columns)

# Recover statistic before exploding
lines_per_station = train_stations.set_index('map_id')['line'].str.count(',')+1

# Explode multi-line stations to keep correct cardinality of lines
train_stations['line'] = train_stations['line'].str.split(',')
train_stations = train_stations.explode('line')

# Agg station -> line
train_line_features = train_stations.groupby('line')[feature_cols].any().astype(float).reset_index()

In [8]:
# Aggregate ridership and merge line-level features:

# The ridership data isn't at directional granularity, even though some stations 
# do have separate directional entrances. We will apportion ridership equally per
# line at multi-line stations.
# XXX: Improve this model of line popularity?
train_line_rides = train_rides.assign(rides = train_rides['rides'] / train_rides['map_id'].map(lines_per_station))
train_line_rides = train_line_rides.groupby(['line','date','dotw','is_weekend','DNC'],as_index=False).agg({'rides':'sum'})
train_line_rides = train_line_rides.merge(train_line_features)
train_line_rides = train_lines.merge(train_line_rides, how='right', on='line')

# Bike Rides + Stations

In [9]:
# # Double-checking if all bike stations (for the years currently accessed)
# #       already have valid (ie chicago?) geometries
# from shapely import box
# chi_boundary = gpd.read_file(CHI_BOUNDARY_FILE)
# chi_bound_geo = chi_boundary['geometry'].to_crs(LOCAL_CRS).iloc[0]
# chi_box = box(*chi_bound_geo.bounds) # Using bounding box for faster computation
# is_valid = bike_rides['geometry'].to_crs(LOCAL_CRS).within(chi_box)

# # "Invalid" geos are all in Evanston essentially, which is fine for our purposes.
# fig = chi_boundary.boundary.plot()
# bike_rides[~is_valid]['geometry'].drop_duplicates().plot(ax=fig, color='red')

In [10]:
print("Columns to gain by merging: ", set(bike_stations.columns) - set(bike_rides.columns))

Columns to gain by merging:  {'station_id'}


XXX:

For now, since all data vintages currently pulled are denormalized and already
have valid geometries, on which we've already computed our spatial features,
there's actually nothing to merge via bike_stations.

If you return to this, note that within bike_stations the station_id <-> geometry is not 1:1,
even though according to the documentation each station has multiple bike docks.
Some of the points are thousands of feet away per station_id.
Therefore be careful about how you construct the merge.

# Bus Rides + Stops + Routes

Dropping stops with unknown tracts (ie outside chicago)

In [11]:
print("Dropping {} ({:.1%}) of rows".format(
    bus_stops[['route','tract']].isna().any(axis=1).sum(),
    bus_stops[['route','tract']].isna().any(axis=1).mean(),
))
bus_stops = bus_stops.dropna(subset=['route','tract'])

Dropping 1 (0.0%) of rows


Note: since the bus ridership is at route granularity, not stop or point granularity,
we need to aggregate to stop-level features to route.

This is tricky since routes span miles across the city. 

## Numeric

My first thought is to 
compute the proportion of stops that exhibit a certain feature, ie proportion
of stops near the United Center. 


In [12]:
bus_stops_features = bus_stops.groupby('route')[feature_cols].mean()

This turns out poorly scaled -- most routes
have zero stops near the United Center, while those that do tend to have a very
small proportion.

In [13]:
print('Proportion of routes with ANY POI stops')
print((bus_stops_features > 0).mean().round(2).to_dict())

Proportion of routes with ANY POI stops
{'airport': 0.07, 'GUARANTEED RATE FIELD 1600m': 0.08, 'GUARANTEED RATE FIELD 400m': 0.03, 'GUARANTEED RATE FIELD 800m': 0.04, 'HYATT REGENCY MCCORMICK PLACE 1600m': 0.1, 'HYATT REGENCY MCCORMICK PLACE 400m': 0.01, 'HYATT REGENCY MCCORMICK PLACE 800m': 0.04, 'SOLDIER FIELD 1600m': 0.18, 'SOLDIER FIELD 400m': 0.03, 'SOLDIER FIELD 800m': 0.12, 'UNITED CENTER 1600m': 0.1, 'UNITED CENTER 400m': 0.03, 'UNITED CENTER 800m': 0.05, 'WRIGLEY FIELD 1600m': 0.12, 'WRIGLEY FIELD 400m': 0.01, 'WRIGLEY FIELD 800m': 0.06}


In [14]:
print("Distribution of mean POI stops per route, given route serves ANY POI's:")
bus_stops_features[(bus_stops_features>0).agg('any',axis=1)].describe()

Distribution of mean POI stops per route, given route serves ANY POI's:


,airport,GUARANTEED RATE FIELD 1600m,GUARANTEED RATE FIELD 400m,GUARANTEED RATE FIELD 800m,HYATT REGENCY MCCORMICK PLACE 1600m,HYATT REGENCY MCCORMICK PLACE 400m,HYATT REGENCY MCCORMICK PLACE 800m,SOLDIER FIELD 1600m,SOLDIER FIELD 400m,SOLDIER FIELD 800m,UNITED CENTER 1600m,UNITED CENTER 400m,UNITED CENTER 800m,WRIGLEY FIELD 1600m,WRIGLEY FIELD 400m,WRIGLEY FIELD 800m
count,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000
mean,0.002678,0.043780,0.002999,0.011403,0.029483,0.000734,0.008595,0.068063,0.005880,0.019525,0.051464,0.003503,0.014437,0.048024,0.001447,0.009333
std,0.006883,0.120862,0.013013,0.041513,0.074510,0.004235,0.034018,0.127768,0.029103,0.063107,0.123457,0.014706,0.044186,0.107180,0.008199,0.025111
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.099010,0.000000,0.006289,0.000000,0.000000,0.000000,0.029412,0.000000,0.000000
max,0.031250,0.725490,0.078431,0.254902,0.468750,0.028571,0.234375,0.600000,0.200000,0.400000,0.666667,0.086614,0.222222,0.642857,0.050000,0.106918


## Binary

Instead we will interpret simply as binary whether or not the route serves the POI
anywhere along the route, without taking into account the lenght of the route,
nor ridership density along the route.

In [15]:
bus_stops_features = bus_stops.groupby('route')[feature_cols].any().astype(float)
bus_routes = bus_routes.merge(bus_stops_features, how='left', left_on='route', right_index=True)

In [16]:
bus_rides = bus_routes.merge(bus_rides, on='route', how='right', validate='1:m')

# Uber Rides + Tracts

XXX:
Not dropping anonymized observations (null tract/comm) because they still add to the non-spatial
time series count.

In [17]:
uber_tracts = tracts[feature_cols + ['geometry','centroid','geoid10']] \
                [lambda x: x['geoid10'].isin(uber_rides[uber_rides.unit == 'tract']['id'])] \
                .rename(columns={'geoid10':'id'})

uber_comms = comms[feature_cols + ['geometry','centroid','comm_area']] \
                [lambda x: x['comm_area'].isin(uber_rides[uber_rides.unit == 'comm']['id'])]\
                .rename(columns={'comm_area':'id'})

assert len(uber_rides.columns.intersection(uber_tracts.columns)) == 1
assert len(uber_rides.columns.intersection(uber_comms.columns)) == 1

uber_tract_rides = uber_tracts.merge(uber_rides[uber_rides.unit == 'tract'], how='right', on='id')
uber_comm_rides = uber_comms.merge(uber_rides[uber_rides.unit == 'comm'], how='right', on='id')
uber_rides = pd.concat([uber_tract_rides, uber_comm_rides], ignore_index=True)

# Pipeline out

In [18]:
uber_rides.to_parquet(uber_rides_out)
bike_rides.to_parquet(bike_rides_out)
train_rides.to_parquet(train_station_rides_out)
train_line_rides.to_parquet(train_line_rides_out)
bus_rides.to_parquet(bus_rides_out)